# Cricket Intelligence Agent - Testing Notebook

Interactive notebook for testing and exploring the Cricket Intelligence Agent.

## Features
- Test cricket stats queries (SQL)
- Test news search queries (Vector DB)
- Test mixed queries
- View reasoning and insights
- Analyze agent performance

## Setup

In [ ]:
import os
import sys
import json
from pathlib import Path

# Add project root to path (notebooks folder is one level down from root)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

from dotenv import load_dotenv
load_dotenv()

print(f"✓ Project root: {PROJECT_ROOT}")
print(f"✓ OpenAI API Key: {'Found' if os.getenv('OPENAI_API') else 'Not Found'}")

## Initialize Agent

In [ ]:
from agent.cricket_agent import CricketAgent

# Create and initialize agent
agent = CricketAgent()
await agent.initialize()

print("\n✓ Agent ready!")

## Test Queries

### Helper Functions

In [ ]:
def print_result(result: dict):
    """Pretty print agent result with reasoning and SQL"""
    print("\n" + "="*60)
    print("RESPONSE:")
    print("="*60)
    print(result['response'])
    
    # Extract and show SQL queries
    insight = result.get('insight', {})
    raw_data = insight.get('raw_data', [])
    
    sql_queries = [r for r in raw_data if r.get('name') == 'execute_sql']
    if sql_queries:
        print("\n" + "-"*60)
        print("SQL QUERIES EXECUTED:")
        print("-"*60)
        for i, sql_result in enumerate(sql_queries, 1):
            content = sql_result.get('content', '')
            try:
                import json
                data = json.loads(content)
                if 'sql' in data:
                    print(f"\n📊 Query {i}:")
                    print(data['sql'])
                    print(f"   → Returned {data.get('row_count', 'unknown')} rows")
                elif 'error' in data:
                    print(f"\n❌ Query {i} Error:")
                    print(data['error'])
            except Exception as e:
                print(f"\nQuery {i}: {content[:200]}...")
    
    # Show agent's reasoning about the results
    messages = result.get('messages', [])
    print("\n" + "-"*60)
    print("AGENT REASONING:")
    print("-"*60)
    
    # Find AIMessages with actual reasoning content (not tool calls)
    for msg in messages:
        if hasattr(msg, 'content') and msg.content:
            # Skip if it's just making tool calls without reasoning
            if hasattr(msg, 'tool_calls') and msg.tool_calls and not msg.content.strip():
                continue
            # Show reasoning from AI
            if type(msg).__name__ == 'AIMessage':
                content = msg.content.strip()
                # Only show if it's substantive reasoning (not the final summary)
                if content and content != result['response'] and len(content) > 50:
                    print(f"\n💭 {content}")
                    break
    
    # Show what tools were used
    tool_calls_found = False
    print("\n" + "-"*60)
    print("TOOLS USED:")
    print("-"*60)
    for msg in messages:
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            if not tool_calls_found:
                tool_calls_found = True
            for tool_call in msg.tool_calls:
                tool_name = tool_call.get('name', 'unknown')
                print(f"  • {tool_name}")
    
    # Show insights
    if insight:
        print("\n" + "-"*60)
        print("KEY INSIGHTS:")
        print("-"*60)
        
        for i, ins in enumerate(insight.get('insights', []), 1):
            print(f"{i}. {ins}")
        
        print(f"\n📌 Query Type: {insight.get('query_type', 'unknown')}")
        print(f"📌 Confidence: {insight.get('confidence', 'medium')}")
    
    print("\n" + "="*60 + "\n")


async def ask(query: str):
    """Ask agent a question"""
    print(f"\n🏏 Query: {query}")
    result = await agent.chat(query)
    print_result(result)
    return result

### Test 1: Cricket Statistics Query

In [ ]:
# Test SQL query
result = await ask("Who are the top 5 run scorers in Test cricket history?")

### Test 2: Cricket News Query

In [ ]:
# Test news search
result = await ask("What's the latest news about Virat Kohli?")

### Test 3: Mixed Query (Stats + News)

In [ ]:
# Test combined query
result = await ask("Tell me about Rohit Sharma's Test cricket stats and recent news")

### Test 4: Complex SQL Query

In [ ]:
# Test more complex query
result = await ask("Which players have scored more than 50 centuries in Test cricket?")

### Test 5: Head-to-Head Stats

In [ ]:
# Test head-to-head query
result = await ask("Show me India vs England head-to-head record in Test cricket")

## Custom Query

In [ ]:
# Try your own query
my_query = "Your question here"
result = await ask(my_query)

## Inspect Raw Data

In [ ]:
# View raw tool results from last query
if result.get('insight', {}).get('raw_data'):
    print("Raw Tool Results:")
    for i, tool_result in enumerate(result['insight']['raw_data'], 1):
        print(f"\nTool {i}: {tool_result.get('name', 'unknown')}")
        try:
            data = json.loads(tool_result.get('content', '{}'))
            print(json.dumps(data, indent=2))
        except:
            print(tool_result.get('content', ''))

## Cleanup

In [ ]:
# Close agent connection
await agent.close()
print("\n✓ Agent closed")